In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import datetime
from matplotlib import style
style.use('ggplot')
import math
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C:/Users/Olatomiwa/Documents/SOL PLAATJE UNIVERSITY/HONOURS 2019/RESEARCH/From Supervisor Dr Mosia/first_yahoo_prices_volumes.csv')

In [3]:
# takes the 7 day percent changes for the prices in the future
def gain_fall(*args): # *args lets us pass any parameters, any number of arguments which becomes an iterable
    cols = [c for c in args] # passing each column mapping it row wise
    for col in cols:
        if(col > 0):
            return(1)   # GAIN
        if(col < 0):
            return(0)# FALL

In [4]:
df['1day_Lag'] = (df['Adj Close'].shift(-1) - df['Adj Close']) / df['Adj Close']
# .shift shifts up to get the future value old - new divided by 
        
df['1day_target'] = list(map(gain_fall, *[df['1day_Lag']]))

df.dropna(inplace = True)
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close,1day_Lag,1day_target
0,2014-09-10,19.680000,19.430000,19.650000,19.610001,4309400.0,18.881136,-0.010199,0.0
1,2014-09-11,19.540001,19.200001,19.469999,19.410000,6268000.0,18.688570,-0.014941,0.0
2,2014-09-12,19.530001,19.100000,19.530001,19.120001,6563400.0,18.409348,-0.013598,0.0
3,2014-09-15,19.209999,18.780001,19.180000,18.860001,7353800.0,18.159008,0.014846,1.0
4,2014-09-16,19.240000,18.750000,18.809999,19.139999,5498400.0,18.428604,0.000523,1.0


In [5]:
df['1day_target'] = df['1day_target'].shift(1)
df['1day_target'].fillna(0, inplace = True)
df['1day_target'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: 1day_target, dtype: float64

In [6]:
# The distribution of the 1day_target mapping
vals = df['1day_target'].values #.tolist optional 
str_vals = [str(i) for i in vals]

print('Data spread: ', Counter(str_vals)) # seeing th way in which buys/sell/hold are distributed

Data spread:  Counter({'1.0': 584, '0.0': 497})


In [7]:
data = {'returns': df['Adj Close'].pct_change(), 'volume': df['Volume'],
                     'y_target': df['1day_target']}
frame = pd.DataFrame(data)
frame.dropna(inplace = True)
frame.head()

,returns,volume,y_target
1,-0.010199,6268000.0,0.0
2,-0.014941,6563400.0,0.0
3,-0.013598,7353800.0,0.0
4,0.014846,5498400.0,1.0
5,0.000523,4339600.0,1.0


# Tensorflow Probability

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import tensorflow_probability as tfp

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
features = frame[['returns', 'volume']]
features.head()

,returns,volume
1,-0.010199,6268000.0
2,-0.014941,6563400.0
3,-0.013598,7353800.0
4,0.014846,5498400.0
5,0.000523,4339600.0


In [11]:
label = frame['y_target']

In [12]:
# Using train_test_fit to create subsets for fitting and testing
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2, random_state = 2, stratify = label)

# Pretend to load synthetic data set.
features = tfp.distributions.Normal(loc=0., scale=1.).sample(int(100e3))
labels = tfp.distributions.Bernoulli(logits=1.618 * features).sample()

In [16]:
model = Sequential([
    # Flattening the input Layer into a vector because it is a matrix
    Flatten(),
    
    Dense(500, activation = 'relu'),
    
    Dropout(0.20),
    
    #output layer with 1 neuron with the sigmoid function to counter for the 2 classes (1 or 0)
    Dense(1, activation = 'sigmoid'), # or tf.nn.softmax
       
    tfp.layers.DenseFlipout(1)
   # tfp.layers.VariationalGaussianProcess(num_inducing_points, kernel_provider) # Bayesian Loss Function
    
])